## Data preproccesing

In [ ]:
import pandas as pd
import datasets

train_df = pd.read_json('../ADL_HW1/datasets/train.json')
valid_df = pd.read_json('../ADL_HW1/datasets/valid.json')
context_df = pd.read_json('../ADL_HW1/datasets/context.json')
# print(train_df.iloc[2])

def reformat(df, have_label=0):
    # Create new columns based on the existing ones
    df['sent1'] = df['question']
    df['sent2'] = ''

    # Extract elements from the 'paragraphs' list into new columns
    df['ending0'] = df['paragraphs'].apply(lambda x: context_df.iloc[x[0], 0])
    df['ending1'] = df['paragraphs'].apply(lambda x: context_df.iloc[x[1], 0])
    df['ending2'] = df['paragraphs'].apply(lambda x: context_df.iloc[x[2], 0])
    df['ending3'] = df['paragraphs'].apply(lambda x: context_df.iloc[x[3], 0])

    # Find the index of 'relevant' in 'paragraphs'
    if have_label:
        df['label'] = df.apply(lambda row: row['paragraphs'].index(row['relevant']), axis=1)
        df.drop(['relevant'], axis=1, inplace=True)

    df.drop(['question', 'paragraphs'], axis=1, inplace=True)
    
    return df

train_df = reformat(train_df, have_label=1)
valid_df = reformat(valid_df, have_label=1)

train_dataset = datasets.Dataset.from_pandas(train_df)
valid_dataset = datasets.Dataset.from_pandas(valid_df)
raw_dataset = datasets.DatasetDict({"train":train_dataset,"validation":valid_dataset})

column_names = raw_dataset["train"].column_names
print('\n', raw_dataset)
print('\n', column_names)

raw_dataset.save_to_disk('./tmp/raw_dataset_PS')

## Fine-tune existing model

In [ ]:
!python ./run_swag_no_trainer.py \
  --model_name_or_path bert-base-chinese \
  --dataset_name ./tmp/raw_dataset_PS \
  --max_seq_length 512 \
  --per_device_train_batch_size 1 \
  --learning_rate 3e-5 \
  --num_train_epochs 2 \
  --gradient_accumulation_steps 2 \
  --output_dir ./PS_model/ \